In [5]:
import os
import re
import requests
from langchain_huggingface import HuggingFaceEmbeddings
from bs4 import BeautifulSoup
from langchain_chroma import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [6]:
web_link='https://www.gamedeveloper.com/design/classic-postmortem-telltale-games-the-walking-dead-2012'

response = requests.get(web_link)
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)

In [7]:
from langchain.text_splitter import CharacterTextSplitter

# Dividir el texto en fragmentos
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,  # Tamaño de cada fragmento
    chunk_overlap=100,  # Superposición entre fragmentos
    length_function=len
)
chunks = text_splitter.split_text(text)


Created a chunk of size 1129, which is longer than the specified 1000


In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


In [11]:
from langchain_chroma import Chroma


vector_store = Chroma.from_texts(
    texts=chunks,
    collection_name="some_facts",
    embedding=embeddings,
    persist_directory="./chroma_some_facts",
)

NameError: name 'textos' is not defined

In [ ]:
# Crear una consulta
query = "What is the postmortem about?"

# Convertir la consulta en embedding
query_embedding = embedding_model.encode(query).tolist()

# Buscar los fragmentos más similares
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5  # Número de fragmentos relevantes
)

# Recuperar documentos relevantes
relevant_chunks = results["documents"][0]
print("Relevant Chunks:", relevant_chunks)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

# Crear el contexto con los fragmentos recuperados
context = "\n".join(relevant_chunks)

# Crear el prompt
prompt = f"Answer the question based on the context below:\n\n{context}\n\nQuestion: {query}\nAnswer:"

# Usar un modelo LLM para generar la respuesta
llm = OpenAI(model_name="gpt-4")
response = llm(prompt)
print("Response:", response)


In [ ]:
client = chromadb.Client(Settings(
    persist_directory="./chromadb_store",
    chroma_db_impl="duckdb+parquet"
))
collection = client.get_collection("web_data")
